In [1]:
# ================================================================
# Single Colab cell: Download images from Drive folder ID, remove balls, save outputs, zip & download
# ================================================================

# Install / upgrade required libs
!pip install -q --upgrade ultralytics opencv-python-headless google-api-python-client google-auth-httplib2 google-auth-oauthlib

import os
import io
import zipfile
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import auth, files
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload
from oauth2client.client import GoogleCredentials

# ---------------- CONFIG ----------------
FOLDER_ID = "1m36zN0wauHii_b8kNk6ORS7dG-F-rcxv"  # <- your Drive folder id
LOCAL_INPUT = "/content/input_images"
LOCAL_OUTPUT = "/content/output_images"
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/ball_removed_results"  # where to save outputs in your Drive
ZIP_PATH = "/content/removed_images.zip"
VALID_EXT = (".jpg", ".jpeg", ".png")

os.makedirs(LOCAL_INPUT, exist_ok=True)
os.makedirs(LOCAL_OUTPUT, exist_ok=True)
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

# ---------------- Authenticate & build Drive service ----------------
print("Authenticating (you will be prompted)...")
auth.authenticate_user()
creds = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=creds, cache_discovery=False)
print("Drive API ready.")

# ---------------- List files in folder ----------------
print(f"Listing files in folder id: {FOLDER_ID} ...")
resp = service.files().list(q=f"'{FOLDER_ID}' in parents and trashed=false",
                            fields="files(id, name, mimeType)").execute()
items = resp.get('files', [])
if not items:
    raise SystemExit("No files found in that Drive folder (or you don't have access).")

# ---------------- Download image files to local input ----------------
downloaded = 0
for f in items:
    name = f['name']
    fid = f['id']
    lname = name.lower()
    if not lname.endswith(VALID_EXT):
        print("Skipping (not image):", name)
        continue
    out_path = os.path.join(LOCAL_INPUT, name)
    print("Downloading:", name)
    request = service.files().get_media(fileId=fid)
    fh = io.FileIO(out_path, mode='wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.close()
    downloaded += 1

print(f"Downloaded {downloaded} images to {LOCAL_INPUT}")
if downloaded == 0:
    raise SystemExit("No images downloaded. Ensure the folder contains JPG/PNG images and you have access.")

# ---------------- Load YOLO model ----------------
print("Loading YOLO model (may download weights)...")
try:
    model = YOLO("yolov8x-seg.pt")
except Exception as e:
    print("Could not load local weights ('yolov8x-seg.pt'), falling back to alias.")
    model = YOLO("yolov8x-seg")  # ultralytics may auto-download

names = getattr(model.model, "names", {})
ball_classes = [k for k,v in names.items() if "ball" in v.lower()]
print("ball_classes (detected by name):", ball_classes)

# ---------------- Helper funcs ----------------
def refine_mask(mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.GaussianBlur(mask, (7,7), 0)
    _, mask = cv2.threshold(mask, 50, 255, cv2.THRESH_BINARY)
    return mask

def hough_fallback(img_gray):
    h, w = img_gray.shape
    minR = max(3, min(h,w)//200)
    maxR = max(12, min(h,w)//25)
    circles = cv2.HoughCircles(img_gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=min(h,w)//10,
                               param1=100, param2=15, minRadius=minR, maxRadius=maxR)
    if circles is None:
        return None
    circles = np.uint16(np.around(circles[0]))
    circles = sorted(circles, key=lambda c: c[2])
    x,y,r = circles[0]
    mask = np.zeros((h,w), dtype=np.uint8)
    cv2.circle(mask, (x,y), r, 255, -1)
    return mask

# ---------------- Process images ----------------
input_files = sorted([f for f in os.listdir(LOCAL_INPUT) if f.lower().endswith(VALID_EXT)])
print("Processing", len(input_files), "images...")

for fname in input_files:
    path = os.path.join(LOCAL_INPUT, fname)
    img = cv2.imread(path)
    if img is None:
        print("Could not read", fname, "- skipping")
        continue
    h, w = img.shape[:2]
    img_area = h*w

    # inference (try to filter by ball class ids if present)
    try:
        if ball_classes:
            res = model(img, classes=ball_classes, conf=0.25, iou=0.45)[0]
        else:
            res = model(img, conf=0.25, iou=0.45)[0]
    except Exception as e:
        print("Inference error with class filter:", e, "-> retrying without class filter")
        res = model(img)[0]

    chosen_mask = None
    if getattr(res, "masks", None) is not None:
        try:
            masks_arr = res.masks.data.cpu().numpy()
        except Exception:
            # fallbacks for API variants
            try:
                masks_arr = np.array([m.numpy() for m in res.masks])
            except Exception:
                masks_arr = None
        if masks_arr is not None:
            best_area = img_area
            for m in masks_arr:
                binm = (m > 0.5).astype(np.uint8)
                area = binm.sum()
                area_ratio = area / img_area
                ys, xs = np.where(binm > 0)
                if xs.size == 0 or ys.size == 0:
                    continue
                aspect_ratio = (np.max(xs)-np.min(xs)) / (np.max(ys)-np.min(ys) + 1e-5)
                if 0.00001 < area_ratio < 0.015 and 0.7 < aspect_ratio < 1.3:
                    if area < best_area:
                        best_area = area
                        chosen_mask = (binm * 255).astype(np.uint8)
            if chosen_mask is not None:
                chosen_mask = cv2.resize(chosen_mask, (w, h), interpolation=cv2.INTER_NEAREST)

    # Hough fallback
    if chosen_mask is None:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_blur = cv2.GaussianBlur(gray, (7,7), 1.5)
        hm = hough_fallback(gray_blur)
        if hm is not None:
            chosen_mask = hm
            print(f"{fname}: Hough fallback used")
        else:
            print(f"{fname}: No ball detected - skipping")
            continue

    mask = refine_mask(chosen_mask)
    inpaint_mask = (mask > 0).astype(np.uint8) * 255

    # Inpaint Telea -> NS
    t1 = cv2.inpaint(img, inpaint_mask, 3, cv2.INPAINT_TELEA)
    t2 = cv2.inpaint(t1, inpaint_mask, 3, cv2.INPAINT_NS)
    out_img = t2

    # small border blend
    border = cv2.Canny(inpaint_mask, 50, 150)
    if border.sum() > 0:
        blurred = cv2.GaussianBlur(out_img, (5,5), 0)
        border3 = np.repeat((border>0)[:,:,None], 3, axis=2)
        out_img = np.where(border3, blurred, out_img)

    out_name = "removed_" + fname
    local_out_path = os.path.join(LOCAL_OUTPUT, out_name)
    drive_out_path = os.path.join(DRIVE_OUTPUT_DIR, out_name)

    cv2.imwrite(local_out_path, out_img)
    # Write to Drive output folder using Drive API (upload) so path is robust
    # First upload local file to Drive_OUTPUT_DIR by creating a file in MyDrive and uploading content
    # We will try writing directly to the mounted Drive path (if available) first:
    try:
        cv2.imwrite(drive_out_path, out_img)
        print(f"{fname}: saved to Drive path {drive_out_path}")
    except Exception as e:
        # Fallback: upload via Drive API into root MyDrive folder with folder id detection
        try:
            # find (or create) folder MyDrive/ball_removed_results and upload file inside it
            # get folder id by searching by name in MyDrive; if not found create it
            q = "name='ball_removed_results' and mimeType='application/vnd.google-apps.folder' and 'root' in parents and trashed=false"
            folder_resp = service.files().list(q=q, fields="files(id, name)").execute()
            folders = folder_resp.get('files', [])
            if folders:
                out_folder_id = folders[0]['id']
            else:
                file_metadata = {'name': 'ball_removed_results', 'mimeType': 'application/vnd.google-apps.folder'}
                created = service.files().create(body=file_metadata, fields='id').execute()
                out_folder_id = created.get('id')
            # Upload file
            media = MediaIoBaseUpload(io.FileIO(local_out_path, 'rb'), mimetype='image/png')
            file_metadata = {'name': out_name, 'parents': [out_folder_id]}
            service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            print(f"{fname}: uploaded to Drive folder id {out_folder_id} via API")
        except Exception as e2:
            print(f"{fname}: failed to save to Drive by both methods: {e} / {e2}")
            # still local file exists

    print(f"{fname}: processed -> {local_out_path}")

# ---------------- Zip outputs and prompt download ----------------
print("Zipping outputs...")
with zipfile.ZipFile(ZIP_PATH, 'w', zipfile.ZIP_DEFLATED) as zf:
    for root, _, files_in in os.walk(LOCAL_OUTPUT):
        for f in files_in:
            zf.write(os.path.join(root, f), arcname=f)

print("Created ZIP:", ZIP_PATH)
print("Starting browser download (you will be prompted)...")
files.download(ZIP_PATH)

print("All done. Processed images are saved locally in", LOCAL_OUTPUT, "and attempts to save to Drive folder:", DRIVE_OUTPUT_DIR)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.41.1 which is incompatible.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Authenticating (you will be prompted)...
Drive API ready.
Listing files in folder id: 1m36zN0wauHii_b8kNk6ORS7dG-F-rcxv ...
Downloading: IMG-20251210-WA0027.jpg
Downloading: IMG-20251210-WA0025.jpg
Downloading: IMG-20251210-WA0026.jpg
Downloading: IMG-2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All done. Processed images are saved locally in /content/output_images and attempts to save to Drive folder: /content/drive/MyDrive/ball_removed_results
